In [1]:
from google.colab import drive

drive.mount('/content/drive')
root_path = 'drive/My Drive/Lab/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from utils import *
from regularization import Regularization as reg
from dataloaders import DataLoaders
from model_with_bn import Net as bn_model
from model_with_gbn import Net as gbn_model
from train import TrainModel
from test import TestModel
from graph_plot import getPlottedGraph
from misclassified_images import MissclassifiedImages as MI

# Train and Test model plot
l1_train_losses = []
l1_train_accuracy = []
l2_train_losses = []
l2_train_accuracy = []
l1_l2_train_losses = []
l1_l2_train_accuracy = []
gbn_train_losses = []
gbn_train_accuracy = []
gbn_l2_train_losses = []
gbn_l2_train_accuracy = []
gbn_l1_l2_train_losses = []
gbn_l1_l2_train_accuracy = []

l1_test_losses = []
l1_test_accuracy = []
l2_test_losses = []
l2_test_accuracy = []
l1_l2_test_losses = []
l1_l2_test_accuracy = []
gbn_test_losses = []
gbn_test_accuracy = []
gbn_l2_test_losses = []
gbn_l2_test_accuracy = []
gbn_l1_l2_test_losses = []
gbn_l1_l2_test_accuracy = []

IMAGE_PATH = "/content/drive/My Drive/Lab/images/"
MODEL_PATH = "/content/drive/My Drive/Lab/models/"
def main():

    # Hyper parameters
    EPOCHS = 25


    # For reproducibility
    SEED = 1
    # Check for CUDA?
    cuda = torch.cuda.is_available()
    print("Cuda is available ?", cuda)
    torch.manual_seed(SEED)
    if cuda:
        torch.cuda.manual_seed(SEED)

    train_dataset, test_dataset, train_loader, test_loader = DataLoaders.dataload()

    device = torch.device("cuda" if cuda else "cpu")

    # Summary
    # summary(model, input_size=(1, 28, 28))

    # Optimizer
    model1 = bn_model().to(device)
    optimizer1 = optim.SGD(model1.parameters(), lr=0.01, momentum=0.9)
    scheduler1 = StepLR(optimizer1, step_size=7, gamma=0.1)

    model2 = gbn_model().to(device)
    optimizer2 = optim.SGD(model2.parameters(), lr=0.01, momentum=0.9)
    scheduler2 = StepLR(optimizer2, step_size=7, gamma=0.1)

    for epoch in range(EPOCHS):
        # With L1
        l1_train_loss, l1_train_acc = TrainModel.train(model1, device, train_loader, optimizer1, epoch,L1_regularization=reg,m_type="L1")
        l1_train_losses.append(l1_train_loss)
        l1_train_accuracy.append(l1_train_acc)
        #scheduler1.step_size = 23
        scheduler1.step()
        l1_test_loss, l1_test_acc = TestModel.test(model1, device, test_loader)
        l1_test_losses.append(l1_test_loss)
        l1_test_accuracy.append(l1_test_acc)

        # With L2
        optimizer1.param_groups[0]['weight_decay'] = 0.0001
        l2_train_loss, l2_train_acc = TrainModel.train(model1, device, train_loader, optimizer1, epoch,m_type="L2")
        l2_train_losses.append(l2_train_loss)
        l2_train_accuracy.append(l2_train_acc)
        #scheduler1.step_size = 3
        scheduler1.step()
        l2_test_loss, l2_test_acc = TestModel.test(model1, device, test_loader)
        l2_test_losses.append(l2_test_loss)
        l2_test_accuracy.append(l2_test_acc)

        # With L1 and L2
        optimizer1.param_groups[0]['weight_decay'] = 0.0001
        l1_l2_train_loss, l1_l2_train_acc = TrainModel.train(model1, device, train_loader, optimizer1, epoch, L1_regularization=reg,m_type="L1&L2")
        l1_l2_train_losses.append(l1_l2_train_loss)
        l1_l2_train_accuracy.append(l1_l2_train_acc)
       # scheduler1.step_size = 19
        scheduler1.step()
        l1_l2_test_loss, l1_l2_test_acc = TestModel.test(model1, device, test_loader)
        l1_l2_test_losses.append(l1_l2_test_loss)
        l1_l2_test_accuracy.append(l1_l2_test_acc)

        # With GBN
        gbn_train_loss, gbn_train_acc = TrainModel.train(model2, device, train_loader, optimizer2, epoch, m_type="GBN")
        gbn_train_losses.append(gbn_train_loss)
        gbn_train_accuracy.append(gbn_train_acc)
       # scheduler2.step_size = 11
        scheduler2.step()
        gbn_test_loss, gbn_test_acc = TestModel.test(model2, device, test_loader)
        gbn_test_losses.append(gbn_test_loss)
        gbn_test_accuracy.append(gbn_test_acc)

        # GBN With L2
        optimizer2.param_groups[0]['weight_decay'] = 0.0001
        gbn_l2_train_loss, gbn_l2_train_acc = TrainModel.train(model2, device, train_loader, optimizer2, epoch,  m_type="GBN&L2")
        gbn_l2_train_losses.append(gbn_l2_train_loss)
        gbn_l2_train_accuracy.append(gbn_l2_train_acc)
       # scheduler2.step_size = 6
        scheduler2.step()
        gbn_l2_test_loss, gbn_l2_test_acc = TestModel.test(model2, device, test_loader)
        gbn_l2_test_losses.append(gbn_l2_test_loss)
        gbn_l2_test_accuracy.append(gbn_l2_test_acc)

        # GBN With L1 and L2
        optimizer2.param_groups[0]['weight_decay'] = 0.0001
        gbn_l1_l2_train_loss, gbn_l1_l2_train_acc = TrainModel.train(model2, device, train_loader, optimizer2, epoch, L1_regularization=reg, m_type="GBN&L1&L2")
        gbn_l1_l2_train_losses.append(gbn_l1_l2_train_loss)
        gbn_l1_l2_train_accuracy.append(gbn_l1_l2_train_acc)
       # scheduler2.step_size = 21
        scheduler2.step()
        gbn_l1_l2_test_loss, gbn_l1_l2_test_acc = TestModel.test(model2, device, test_loader)
        gbn_l1_l2_test_losses.append(gbn_l1_l2_test_loss)
        gbn_l1_l2_test_accuracy.append(gbn_l1_l2_test_acc)

    #Save Models
    #PATH = "/content/drive/My Drive/Lab/Loss_and_accuracy_plot.png"
    torch.save(model1, MODEL_PATH+"bn_model.pth")
    torch.save(model2, MODEL_PATH+"gbn_model.pth")

    #Plot and save graph of losses and accuracy
    getPlottedGraph(EPOCHS, l1_train_losses, l1_train_accuracy, l1_test_losses, l1_test_accuracy,
                    l2_train_losses, l2_train_accuracy,l2_test_losses, l2_test_accuracy,
                    l1_l2_train_losses, l1_l2_train_accuracy,l1_l2_test_losses, l1_l2_test_accuracy,
                    gbn_train_losses, gbn_train_accuracy, gbn_test_losses, gbn_test_accuracy,
                    gbn_l2_train_losses, gbn_l2_train_accuracy, gbn_l2_test_losses, gbn_l2_test_accuracy,
                    gbn_l1_l2_train_losses, gbn_l1_l2_train_accuracy, gbn_l1_l2_test_losses, gbn_l1_l2_test_accuracy,name="plot", PATH=IMAGE_PATH)

    #Save misclassified images
    MI.show_save_misclassified_images(model1, test_loader, name="fig1",PATH=IMAGE_PATH, max_misclassified_imgs=25)
    MI.show_save_misclassified_images(model2, test_loader, name="fig2",PATH=IMAGE_PATH, max_misclassified_imgs=25)
if __name__ == "__main__":
    main()


Cuda is available ? True


/content/ghost_batchnormalization.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(self.weight.data.fill_(1.0)).cuda()
/content/ghost_batchnormalization.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(self.bias.data.fill_(0.0)).cuda()
L1: EPOCH= 0 Loss= 0.4605 Batch_id= 468 Accuracy= 87.30: 100%|██████████| 469/469 [00:16<00:00, 29.18it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average Loss= 0.0938 Batch_id= 78 Accuracy= 9761/10000 (97.61%)



L2: EPOCH= 0 Loss= 0.1651 Batch_id= 468 Accuracy= 97.34: 100%|██████████| 469/469 [00:14<00:00, 33.30it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average Loss= 0.0460 Batch_id= 78 Accuracy= 9863/10000 (98.63%)



...


Test set: Average Loss= 0.0212 Batch_id= 78 Accuracy= 9938/10000 (99.38%)



GBN&L1&L2: EPOCH= 18 Loss= 0.3957 Batch_id= 468 Accuracy= 98.93: 100%|██████████| 469/469 [00:15<00:00, 29.88it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average Loss= 0.0211 Batch_id= 78 Accuracy= 9941/10000 (99.41%)



L1: EPOCH= 19 Loss= 0.2380 Batch_id= 429 Accuracy= 99.13:  91%|█████████ | 427/469 [00:14<00:01, 31.87it/s]

Buffered data was truncated after reaching the output size limit.

In [ ]:
# getPlottedGraph(25, l1_train_losses, l1_train_accuracy, l1_test_losses, l1_test_accuracy,
#                     l2_train_losses, l2_train_accuracy,l2_test_losses, l2_test_accuracy,
#                     l1_l2_train_losses, l1_l2_train_accuracy,l1_l2_test_losses, l1_l2_test_accuracy,
#                     gbn_train_losses, gbn_train_accuracy, gbn_test_losses, gbn_test_accuracy,
#                     gbn_l1_l2_train_losses, gbn_l1_l2_train_accuracy, gbn_l1_l2_test_losses, gbn_l1_l2_test_accuracy)

In [3]:

print(max(l1_train_accuracy))
print(max(l1_test_accuracy))
print(l1_test_accuracy.index(max(l1_test_accuracy)))
print("********************************************************************************************************")
print(max(l2_train_accuracy))
print(max(l2_test_accuracy))
print(l2_test_accuracy.index(max(l2_test_accuracy)))
print("********************************************************************************************************")
print(max(l1_l2_train_accuracy))
print(max(l1_l2_test_accuracy))
print(l1_l2_test_accuracy.index(max(l1_l2_test_accuracy)))
print("********************************************************************************************************")
print(max(gbn_train_accuracy))
print(max(gbn_test_accuracy))
print(gbn_test_accuracy.index(max(gbn_test_accuracy)))
print("********************************************************************************************************")
print(max(gbn_l2_train_accuracy))
print(max(gbn_l2_test_accuracy))
print(gbn_l2_test_accuracy.index(max(gbn_l2_test_accuracy)))
print("********************************************************************************************************")
print(max(gbn_l1_l2_train_accuracy))
print(max(gbn_l1_l2_test_accuracy))
print(gbn_l1_l2_test_accuracy.index(max(gbn_l1_l2_test_accuracy)))
print("********************************************************************************************************")


99.14166666666667
99.44
10
********************************************************************************************************
99.135
99.49
5
********************************************************************************************************
99.13166666666666
99.45
5
********************************************************************************************************
99.01
99.44
13
********************************************************************************************************
99.04833333333333
99.44
9
********************************************************************************************************
99.01166666666667
99.45
10
********************************************************************************************************


In [ ]:
# 95.48833333333333
# 97.86
# 23
# ********************************************************************************************************
# 96.76833333333333
# 98.82
# 3
# ********************************************************************************************************
# 96.21833333333333
# 98.13
# 19
# ********************************************************************************************************
# 98.175
# 99.3
# 11
# ********************************************************************************************************
# 98.67333333333333
# 99.46
# 6
# ********************************************************************************************************
# 97.42666666666666
# 98.34
# 21